In [22]:
import os
import sys

# Set up the Python path properly
current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)  # Add the parent directory to sys.path

import numpy as np
import torch

# Update imports to use relative paths or fix absolute paths
from src.sample import sample, save_image  # Use relative import from parent directory
from src.generator.model import Generator  # Use proper path
from src.text_encoder.model import RNNEncoder
from src.utils import create_loader

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
generator = Generator(n_channels=32, latent_dim=100).to(device)
generator.load_state_dict(torch.load("D:\GAN-codes\gen_weights\gen_87.pth", map_location=device))
generator = generator.eval()

C:\Users\nanda\AppData\Local\Temp\ipykernel_3424\1276995260.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load("D:\GAN-codes\gen_weight

In [25]:
train_loader = create_loader(256, 24, "../data", "test")

Total filenames: 11788
Load captions from: ../data\captions.pickle
Load file names from: ../data\test\filenames.pickle (2933)
Dictionary size: 5450
Embeddings number: 10


In [26]:
n_words = train_loader.dataset.n_words

In [27]:
text_encoder = RNNEncoder.load(r"D:\GAN-codes\Deep_Fusion_GAN\text_encoder_weights\text_encoder.pth", n_words)
text_encoder.to(device)

for p in text_encoder.parameters():
    p.requires_grad = False
text_encoder = text_encoder.eval()

c:\Users\nanda\miniconda3\envs\pytorch\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
d:\GAN-codes\Deep_Fusion_GAN\src\text_encoder\model.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We r

In [28]:
%%time
sample(generator, text_encoder, next(iter(train_loader)), "../gen_images")

CPU times: total: 1.03 s
Wall time: 1.06 s


# Own birds

In [29]:
dataset = train_loader.dataset

In [30]:
def gen_own_bird(word_caption, name):
    codes = [dataset.word2code[w] for w in word_caption.lower().split()]
    
    caption = np.array(codes)
    pad_caption = np.zeros((18, 1), dtype='int64')

    if len(caption) <= 18:
        pad_caption[:len(caption), 0] = caption
        len_ = len(caption)
    else:
        indices = list(np.arange(len(caption)))
        np.random.shuffle(indices)
        pad_caption[:, 0] = caption[np.sort(indices[:18])]
        len_ = 18

    # Move tensors to the same device as the model (device global variable)
    embed = text_encoder(torch.tensor(pad_caption, device=device).reshape(1, -1), 
                          torch.tensor([len_], device=device))
    batch_size = embed.shape[0]
    noise = torch.randn(batch_size, 100).to(device)
    img = generator(noise, embed)
    save_image(img[0].data.cpu().numpy(), "../gen_images", name)

In [43]:
caption = "A black and white bird with a long beak and a yellow belly"
gen_own_bird(caption, caption)